In [8]:
#importing required libraries
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import pandas_datareader.data as web
import datetime as dt
import pandas as pd
import numpy as np
import math

In [9]:
# Only thing you should set
ticker = 'ATVI'

start = dt.datetime(2016, 1, 1)
#end = dt.datetime(2019, 1, 11)
end = dt.datetime.today()

df = web.DataReader(ticker, 'yahoo', start, end)  # Pick ticker symbol
df.tail()  # Show the last 5

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-10-10,53.840000,51.450001,52.930000,53.689999,8700900.0,53.689999
2019-10-11,55.380001,54.150002,54.349998,54.820000,8627100.0,54.820000
2019-10-14,55.200001,54.009998,54.349998,55.009998,5936000.0,55.009998
2019-10-15,55.599998,54.160000,55.150002,55.560001,6826700.0,55.560001
2019-10-16,55.939800,55.139999,55.400002,55.580002,2563080.0,55.580002


In [11]:
dataset = df.values

In [12]:
train = dataset[0:987,:]
valid = dataset[987:,:]

In [13]:
#converting dataset into x_train and y_train
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

In [14]:
x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

In [15]:
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

In [16]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))

In [25]:
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2)

Epoch 1/1
 - 44s - loss: 0.0017


In [27]:
#predicting 246 values, using past 60 from the train data
inputs = dataset[len(dataset) - len(valid) - 60:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [20]:
X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)

In [21]:
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

ValueError: non-broadcastable output operand with shape (60,1) doesn't match the broadcast shape (60,6)

In [22]:
rms=np.sqrt(np.mean(np.power((valid-closing_price),2)))

ValueError: operands could not be broadcast together with shapes (0,6) (60,1) 

In [23]:
#for plotting
train = new_data[:987]
valid = new_data[987:]
valid['Predictions'] = closing_price
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])

Exception: Data must be 1-dimensional